# Parse Actuals

Looks through the URLs to grab batting & pitching actuals and deliver those back to the user.

In [35]:
import requests
import csv
import datetime
from bs4 import BeautifulSoup

In [74]:
# static urls
season = datetime.datetime.now().year
PITCHERS_URL = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=pit&lg=al&qual=y&type=c,36,37,38,40,-1,120,121,217,-1,24,41,42,43,44,-1,117,118,119,-1,6,45,124,-1,62,122,13&season={season}&month=0&season1={season}&ind=0&team=0&rost=0&age=0&filter=&players=0&page=1_100000".format(season=season)
BATTERS_URL = "https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=al&qual=y&type=8&season={season}&month=0&season1={season}&ind=0&team=0&rost=0&age=0&filter=&players=0&page=1_10000".format(season=season)

# # request the data
pitchers_html = requests.get(PITCHERS_URL).text
batters_html = requests.get(BATTERS_URL).text

Now take the requests and parse out the relevant header information for each of the positions. This function will take one of the fangraphs pages as input and write out a CSV of that information once it's parsed.

In [77]:
def parse_array_from_fangraphs_html(input_html, out_file_name):
    """
    Take a HTML stats page from fangraphs and parse it out to a CSV file.
    """
    # parse input
    soup = BeautifulSoup(input_html, "lxml")
    table = soup.find("table", {"class": "rgMasterTable"})
    
    # get headers
    headers_html = table.find("thead").find_all("th")
    headers = []
    for header in headers_html:
        headers.append(header.text)
    print(headers)
    
    # get rows
    rows = []
    rows_html = table.find("tbody").find_all("tr")
    for row in rows_html:
        row_data = []
        for cell in row.find_all("td"):
            row_data.append(cell.text)
        rows.append(row_data)
    
    # write to CSV file
    with open(out_file_name, "w") as out_file:
        writer = csv.writer(out_file)
        writer.writerow(headers)
        writer.writerows(rows)
        


Now that we have all of the player data, I'm writing these out to a CSV file if I want to load them again later without having to run the requests to those pages once more.

In [78]:
parse_array_from_fangraphs_html(batters_html, 'batters_actuals.csv')
parse_array_from_fangraphs_html(pitchers_html, 'pitchers_actuals.csv')

['#', 'Name', 'Team', 'G', 'PA', 'HR', 'R', 'RBI', 'SB', 'BB%', 'K%', 'ISO', 'BABIP', 'AVG', 'OBP', 'SLG', 'wOBA', 'wRC+', 'BsR', 'Off', 'Def', 'WAR']
['#', 'Name', 'Team', 'K/9', 'BB/9', 'K/BB', 'HR/9', 'K%', 'BB%', 'K-BB%', 'SO', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'ERA-', 'FIP-', 'xFIP-', 'ERA', 'FIP', 'E-F', 'xFIP', 'SIERA', 'IP']


Load those CSV files using read_csv() in pandas. Since some of the percentage values are stored as strings, we need to parse those into floats.

In [83]:
bdf = pd.read_csv('batters_actuals.csv')
pdf = pd.read_csv('pitchers_actuals.csv')

# create a function to parse out percentage strings to floats
def parse_pctg(value):
    return float(value.split()[0]) / 100

In [84]:
# apply that to all percentage values in the dataframes
bdf['BB%'] = bdf['BB%'].apply(lambda x: parse_pctg(x))
bdf['K%'] = bdf['K%'].apply(lambda x: parse_pctg(x))
pdf['K%'] = pdf['K%'].apply(lambda x: parse_pctg(x))
pdf['BB%'] = pdf['BB%'].apply(lambda x: parse_pctg(x))
pdf['K-BB%'] = pdf['K-BB%'].apply(lambda x: parse_pctg(x))
pdf['LOB%'] = pdf['LOB%'].apply(lambda x: parse_pctg(x))

In [99]:
with open('batters_projections.html', 'r') as bhtml:
    btxt = bhtml.read()
    bpdf = pd.read_html(btxt)[-1]  # read_html returns ALL tables, we just want the last one.
    bpdf.dropna(axis=1, inplace=True)

In [100]:
bpdf.head()

,Name,G,PA,AB,H,2B,3B,HR,R,RBI,...,CS,AVG,OBP,SLG,OPS,wOBA,Fld,BsR,WAR,ADP
0,Mike Trout,81,342,273,83,16,3,21,61,53,...,3,0.303,0.436,0.609,1.045,0.430,0.0,1.8,4.7,1.1
1,Joey Votto,83,349,280,83,16,1,13,48,46,...,1,0.295,0.428,0.499,0.927,0.394,0.9,-1.4,2.3,19.2
2,Giancarlo Stanton,81,342,297,80,14,1,28,55,68,...,1,0.268,0.357,0.603,0.960,0.393,1.6,-0.2,2.7,8.1
3,J.D. Martinez,80,334,298,88,18,1,22,50,61,...,1,0.294,0.364,0.586,0.950,0.393,-0.5,-0.9,2.2,22.8
4,Freddie Freeman,83,350,296,88,20,2,16,51,51,...,3,0.297,0.398,0.541,0.939,0.391,1.9,0.0,2.5,21.1
